In [101]:

import tensorflow as tf
import os
import random
import numpy as np
from keras.models import Sequential
from tqdm import tqdm 
from keras.optimizers import *
from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
from keras.layers import UpSampling2D
from keras.layers import Dense, Concatenate
from keras.preprocessing.image import ImageDataGenerator 

from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np

In [82]:
trainning='dataset/trainning'
mask='dataset/mask'
test='dataset/test'
mask_test='dataset/mask_test'
image_color_mode = "grayscale"
mask_color_mode = "grayscale"
flag_multi_class = False
num_class = 2
save_to_dir = None
target_size = (400,400)
seed = 1
batch_size = 4

In [120]:
def build_unet_model():

    inputs= layers.Input(shape=(400,400,1))
    
    conv1 = tf.keras.layers.Conv2D(64,(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    conv1 = tf.keras.layers.BatchNormalization()(conv1)
    conv1 = tf.keras.layers.Conv2D(64,(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv1)
    conv1 = tf.keras.layers.BatchNormalization()(conv1)
    pool1 = tf.keras.layers.MaxPooling2D((2, 2))(conv1)
      
    conv2 = tf.keras.layers.Conv2D(128,(3, 3),activation='relu', kernel_initializer='he_normal', padding='same')(pool1)
    conv2 = tf.keras.layers.BatchNormalization()(conv2)
    conv2 = tf.keras.layers.Conv2D(128, (3, 3),activation='relu', kernel_initializer='he_normal', padding='same')(conv2)
    conv2 = tf.keras.layers.BatchNormalization()(conv2)
    pool2 = tf.keras.layers.MaxPooling2D((2, 2))(conv2)

    conv3 = tf.keras.layers.Conv2D(256,(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool2)
    conv3 = tf.keras.layers.BatchNormalization()(conv3)
    conv3 = tf.keras.layers.Conv2D(256,(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv3)
    conv3 = tf.keras.layers.BatchNormalization()(conv3)
    pool3 = tf.keras.layers.MaxPooling2D((2, 2))(conv3)

    conv4 = tf.keras.layers.Conv2D(512,(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool3)
    conv4 = tf.keras.layers.BatchNormalization()(conv4)
    conv4 = tf.keras.layers.Conv2D(512,(3, 3),activation='relu', kernel_initializer='he_normal', padding='same')(conv4)
    conv4 = tf.keras.layers.BatchNormalization()(conv4)
    drop4 = tf.keras.layers.Dropout(0.5)(conv4)
    pool4 = tf.keras.layers.MaxPooling2D((2, 2))(conv4)

    conv5 = tf.keras.layers. Conv2D(1024, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool4)
    conv5 = tf.keras.layers.BatchNormalization()(conv5)
    conv5 = tf.keras.layers.Conv2D(1024, (3, 3),activation='relu', kernel_initializer='he_normal', padding='same')(conv5)
    conv5 = tf.keras.layers.BatchNormalization()(conv5)
    drop5 = tf.keras.layers.Dropout(0.5)(conv5)
    #Expansive path 

    u6    = tf.keras.layers.Conv2DTranspose(512,(2, 2), strides=(2, 2), padding='same')(conv5)
    u6    = tf.keras.layers.concatenate([u6,conv4])
    conv6 = tf.keras.layers.Conv2D(512,(3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    conv6 = tf.keras.layers.BatchNormalization()(conv6)
    conv6 = tf.keras.layers.Conv2D(512,(3,3), activation='relu', kernel_initializer='he_normal', padding='same')(conv6)
    conv6 = tf.keras.layers.BatchNormalization()(conv6)
    conv6 = tf.keras.layers.BatchNormalization()(conv6)
        
    u7    = tf.keras.layers.Conv2DTranspose(256,(2, 2), strides=(2, 2), padding='same')(conv6)
    u7    = tf.keras.layers.concatenate([u7,conv3])
    conv7 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    conv7 = tf.keras.layers.BatchNormalization()(conv7)
    conv7 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv7)
    conv7 = tf.keras.layers.BatchNormalization()(conv7)
    conv7 = tf.keras.layers.BatchNormalization()(conv7)
    
    u8    = tf.keras.layers.Conv2DTranspose(128,(2, 2), strides=(2, 2), padding='same')(conv7)
    u8    = tf.keras.layers.concatenate([u8 , conv2])
    conv8 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    conv8 = tf.keras.layers.BatchNormalization()(conv8)
    conv8 = tf.keras.layers. Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv8)
    conv8 = tf.keras.layers.BatchNormalization()(conv8)

    u9    = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    u9    = tf.keras.layers.concatenate([u9 , conv1], axis = 3)
    conv9 = tf.keras.layers.Conv2D(64,(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    conv9 = tf.keras.layers. BatchNormalization()(conv9)
    conv9 = tf.keras.layers.Conv2D(64,(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv9)
    conv9 = tf.keras.layers.BatchNormalization()(conv9)
    conv9 = tf.keras.layers.Conv2D(2,(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv9)
    conv9 = tf.keras.layers.BatchNormalization()(conv9)
    
    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(conv9)  
    unet_model = tf.keras.Model(inputs, outputs, name="U-Net")

    return unet_model

In [121]:
unet_model = unet()

In [122]:
unet_model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, 400, 400, 1) 0                                            
__________________________________________________________________________________________________
conv2d_268 (Conv2D)             (None, 400, 400, 64) 640         input_18[0][0]                   
__________________________________________________________________________________________________
batch_normalization_182 (BatchN (None, 400, 400, 64) 256         conv2d_268[0][0]                 
__________________________________________________________________________________________________
conv2d_269 (Conv2D)             (None, 400, 400, 64) 36928       batch_normalization_182[0][0]    
_______________________________________________________________________________________

In [123]:
def adjustData(img,mask,flag_multi_class,num_class):
    if(flag_multi_class):
        img = img / 255.
        mask = mask[:,:,:,0] if(len(mask.shape) == 4) else mask[:,:,0]
        new_mask = np.zeros(mask.shape + (num_class,))
        for i in range(num_class):
            #for one pixel in the image, find the class in mask and convert it into one-hot vector
            #index = np.where(mask == i)
            #index_mask = (index[0],index[1],index[2],np.zeros(len(index[0]),dtype = np.int64) + i) if (len(mask.shape) == 4) else (index[0],index[1],np.zeros(len(index[0]),dtype = np.int64) + i)
            #new_mask[index_mask] = 1
            new_mask[mask == i,i] = 1
        new_mask = np.reshape(new_mask,(new_mask.shape[0],new_mask.shape[1]*new_mask.shape[2],new_mask.shape[3])) if flag_multi_class else np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
        mask = new_mask
    elif(np.max(img) > 1):
        img = img / 255.
        mask = mask /255.
        mask[mask > 0.5] = 1
        mask[mask <= 0.5] = 0
    return (img,mask)
def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict,image_color_mode = "grayscale",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,target_size = (400,400),seed = 1):
   
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        yield (img,mask)
        
def testGenerator(test_path,num_image = 25,target_size = (400,400),flag_multi_class = False,as_gray = True):
    for i in range(num_image):
        img = io.imread(os.path.join(test_path,"%d.png"%(i+1)),as_gray = as_gray)
        #img = img / 255.
        img = trans.resize(img,target_size)
        img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
        img = np.reshape(img,(1,)+img.shape)
        yield img



In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255
validation_generator = test_datagen.flow_from_directory(
        'dataset',
        classes = ['test'],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        seed = seed)

In [70]:
data_gen_args = dict()
myGene = trainGenerator(4,'dataset','trainning','mask',data_gen_args,save_to_dir = None)
model_history = unet_model.fit(myGene,
                               steps_per_epoch=37,
                               epochs=15)

Found 75 images belonging to 1 classes.
Found 75 images belonging to 1 classes.


ValueError: ZipFile requires mode 'r', 'w', 'x', or 'a'

In [75]:
testGene = testGenerator("dataset/Test")

unet_model.load_weights("dataset/weight/2/last_checkpoint.hdf5")
results = unet_model.predict_generator(testGene,51,verbose=1)
#predicted_image_binary = results > 0.5
saveResult("dataset",results)

ValueError: You are trying to load a weight file containing 19 layers into a model with 23 layers.